In [ ]:
import pandas as pd

In [ ]:
df = pd.read_html('https://fbref.com/en/squads/19538871/2013-2014/matchlogs/c9/schedule/Manchester-United-Scores-and-Fixtures-Premier-League',
                  attrs={"id":"matchlogs_for"})[0]

In [ ]:
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
# Dropping irrelevant columns #
columns_to_drop = ['Time', 'Captain', 'Formation', 'Attendance', 'Opp Formation', 'Referee', 'Match Report', 'Notes']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Dropping rows with missing essential info #
df.dropna(subset=['Date', 'Opponent', 'Result'], inplace=True)

# Extract matchday number from 'Round' column (e.g., "Matchweek 5" → 5) #
df['matchday'] = df['Round'].str.extract(r'(\d+)').astype(int)

# Create points_won column based on results #
result_to_points = {'W': 3, 'D': 1, 'L': 0}
df['points_won'] = df['Result'].map(result_to_points)

# Standardize home/away column to binary values #
df['home_0_away_1'] = df['Venue'].apply(lambda x: 0 if x == 'Home' else 1)

# Reorder columns#
df = df[['matchday', 'Date', 'home_0_away_1', 'Opponent', 'Result', 'points_won', 'GF', 'GA']]

In [ ]:
df

In [ ]:
df['date'] = pd.to_datetime(df['Date'],dayfirst=True)
df = df.sort_values('date')
df['days_since_last_game_united'] = (
    df['date']
      .diff()               # current_date - previous_date
      .dt.days              # convert to integer days
      .fillna(0)            # first match has no “previous” → 0
      .astype(int)
)
df.drop(columns='date', inplace=True, errors='ignore')
df

In [ ]:
df_elo = pd.read_csv('/content/EloRatings.csv')

In [ ]:
df_elo['date'] = pd.to_datetime(df_elo['date'], dayfirst=True)
#filter to only England#
df_elo = df_elo[df_elo['country'] == 'ENG']
#filter to dates on or after July 1, 2013 #
df_elo['date'] = pd.to_datetime(df_elo['date'], format='%Y-%m-%d')
cutoff = pd.Timestamp('2013-07-01')
df_elo = df_elo[df_elo['date'] >= cutoff].reset_index(drop=True)
print(df_elo.country.unique())    # should be just ['ENG']
print(df_elo.date.min(), df_elo.date.max())

In [ ]:
df_elo

In [ ]:
def round_to_elo_date(dt):
    return pd.Timestamp(dt.year, dt.month, 1) if dt.day < 15 else pd.Timestamp(dt.year, dt.month, 15)
df['merge_date'] = df['Date'].apply(round_to_elo_date)
df_elo['merge_date'] = df_elo['date']
name_map = {
    'Arsenal': 'Arsenal',
    'Aston Villa': 'Aston Villa',
    'Cardiff City': 'Cardiff',
    'Chelsea': 'Chelsea',
    'Crystal Palace': 'Crystal Palace',
    'Everton': 'Everton',
    'Fulham': 'Fulham',
    'Hull City': 'Hull',
    'Liverpool': 'Liverpool',
    'Manchester City': 'Man City',
    'Newcastle Utd': 'Newcastle',
    'Norwich City': 'Norwich',
    'Southampton': 'Southampton',
    'Stoke City': 'Stoke',
    'Sunderland': 'Sunderland',
    'Swansea City': 'Swansea',
    'Tottenham': 'Tottenham',
    'West Brom': 'West Brom',
    'West Ham': 'West Ham',
}
df['elo_opponent'] = df['Opponent'].map(name_map)
df['elo_club'] = 'Man United'
df = df.merge(
    df_elo[['club', 'merge_date', 'elo']].rename(columns={'elo': 'opp_elo'}),
    left_on=['elo_opponent', 'merge_date'],
    right_on=['club', 'merge_date'],
    how='left'
).drop(columns='club')
df = df.merge(
    df_elo[['club', 'merge_date', 'elo']].rename(columns={'elo': 'united_elo'}),
    left_on=['elo_club', 'merge_date'],
    right_on=['club', 'merge_date'],
    how='left'
).drop(columns=['club', 'elo_club'])

In [ ]:
df.drop(columns=['merge_date','elo_opponent',], inplace=True, errors='ignore')
df

In [ ]:
print((df["Opponent"].unique()))

In [ ]:
df["elo_diff"] = df["united_elo"] - df["opp_elo"]
final_pos = {
    'Manchester City': 1,
    'Liverpool': 2,
    'Chelsea': 3,
    'Arsenal': 4,
    'Everton': 5,
    'Tottenham': 6,
    'Manchester United': 7,
    'Southampton': 8,
    'Stoke City': 9,
    'Newcastle Utd': 10,
    'Crystal Palace': 11,
    'Swansea City': 12,
    'West Ham': 13,
    'Sunderland': 14,
    'Aston Villa': 15,
    'Hull City': 16,
    'West Brom': 17,
    'Norwich City': 18,
    'Fulham': 19,
    'Cardiff City': 20
}
df['opp_final_pos'] = df['Opponent'].map(final_pos)

In [ ]:
df

In [ ]:
df_bet = pd.read_csv('/content/13-14Odds.csv')
df_bet['Date'] = pd.to_datetime(df_bet['Date'], dayfirst=True)

In [ ]:
df['date'] = pd.to_datetime(df['Date'], dayfirst=True)
home_odds_map = df_bet.set_index(['Date','HomeTeam'])['B365H'].to_dict()
away_odds_map = df_bet.set_index(['Date','AwayTeam'])['B365A'].to_dict()

def get_united_win_odds(row):
    key = (row['date'], 'Man United')
    if row['home_0_away_1'] == 0:
        return home_odds_map.get(key, pd.NA)
    else:
        return away_odds_map.get(key, pd.NA)

df['united_win_odds'] = df.apply(get_united_win_odds, axis=1)

In [ ]:
df.drop(columns='date', inplace=True, errors='ignore')
df

In [ ]:
df["days_since_last_opp_game"] = [0, 8, 8, 14, 8, 7, 6, 13, 7, 7, 8, 15, 7, 4, 3, 7, 7, 5, 2, 3, 10, 8, 10, 3, 8, 4, 14, 14, 15, 7, 3, 6, 7, 4, 6, 6, 3, 8]
df

In [ ]:
df = df[['matchday','Date','home_0_away_1','Opponent','Result','points_won','GF','GA','united_elo', 'opp_elo', 'elo_diff', 'united_win_odds', 'opp_final_pos', 'days_since_last_game_united', 'days_since_last_opp_game']]

In [ ]:
#Final DF for 13/14 season data #
df

,matchday,Date,home_0_away_1,Opponent,Result,points_won,GF,GA,united_elo,opp_elo,elo_diff,united_win_odds,opp_final_pos,days_since_last_game_united,days_since_last_opp_game
0,1,2013-08-17,1,Swansea City,W,3,4,1,1911.40,1665.55,245.85,2.00,12,0,0
1,2,2013-08-26,0,Chelsea,D,1,0,0,1911.40,1887.22,24.18,2.45,3,9,8
2,3,2013-09-01,1,Liverpool,L,0,0,1,1921.33,1801.01,120.32,2.88,2,6,8
3,4,2013-09-14,0,Crystal Palace,W,3,2,0,1921.33,1525.75,395.58,1.22,11,13,14
4,5,2013-09-22,1,Manchester City,L,0,1,4,1911.61,1866.28,45.33,3.60,1,8,8
5,6,2013-09-28,0,West Brom,L,0,1,2,1911.61,1654.51,257.10,1.30,17,6,7
6,7,2013-10-05,1,Sunderland,W,3,2,1,1900.68,1614.09,286.59,1.65,14,7,6
7,8,2013-10-19,0,Southampton,D,1,1,1,1907.86,1682.13,225.73,1.57,8,14,13
8,9,2013-10-26,0,Stoke City,W,3,3,2,1907.86,1658.97,248.89,1.33,9,7,7
9,10,2013-11-02,1,Fulham,W,3,3,1,1909.41,1671.13,238.28,1.57,19,7,7


In [ ]:
df_2 = pd.read_html('https://fbref.com/en/squads/19538871/2014-2015/matchlogs/c9/schedule/Manchester-United-Scores-and-Fixtures-Premier-League', attrs= {"id":"matchlogs_for"})[0]

In [ ]:
df_2.drop(columns = ['Attendance','Poss','Captain','Formation','Opp Formation','Referee', 'Match Report', 'Notes','Day'], inplace=True, errors='ignore')
df_2

,Date,Time,Round,Venue,Result,GF,GA,Opponent
0,2014-08-16,12:45,Matchweek 1,Home,L,1,2,Swansea City
1,2014-08-24,16:00,Matchweek 2,Away,D,1,1,Sunderland
2,2014-08-30,12:45,Matchweek 3,Away,D,0,0,Burnley
3,2014-09-14,16:00,Matchweek 4,Home,W,4,0,QPR
4,2014-09-21,13:30,Matchweek 5,Away,L,3,5,Leicester City
5,2014-09-27,15:00,Matchweek 6,Home,W,2,1,West Ham
6,2014-10-05,12:00,Matchweek 7,Home,W,2,1,Everton
7,2014-10-20,20:00,Matchweek 8,Away,D,2,2,West Brom
8,2014-10-26,16:00,Matchweek 9,Home,D,1,1,Chelsea
9,2014-11-02,13:30,Matchweek 10,Away,L,0,1,Manchester City


In [ ]:
df_2['Date'] = pd.to_datetime(df_2['Date'], dayfirst=True)
# Extract matchday number from 'Round' column (e.g., "Matchweek 5" → 5) #
df_2['matchday'] = df_2['Round'].str.extract(r'(\d+)').astype(int)
# Create points_won column based on results #
result_to_points = {'W': 3, 'D': 1, 'L': 0}
df_2['points_won'] = df_2['Result'].map(result_to_points)
# Standardize home/away column to binary values #
df_2['home_0_away_1'] = df_2['Venue'].apply(lambda x: 0 if x == 'Home' else 1)
# Reorder columns#
df_2 = df_2[['matchday', 'Date', 'home_0_away_1', 'Opponent', 'Result', 'points_won', 'GF', 'GA']]

In [ ]:
df_2

,matchday,Date,home_0_away_1,Opponent,Result,points_won,GF,GA
0,1,2014-08-16,0,Swansea City,L,0,1,2
1,2,2014-08-24,1,Sunderland,D,1,1,1
2,3,2014-08-30,1,Burnley,D,1,0,0
3,4,2014-09-14,0,QPR,W,3,4,0
4,5,2014-09-21,1,Leicester City,L,0,3,5
5,6,2014-09-27,0,West Ham,W,3,2,1
6,7,2014-10-05,0,Everton,W,3,2,1
7,8,2014-10-20,1,West Brom,D,1,2,2
8,9,2014-10-26,0,Chelsea,D,1,1,1
9,10,2014-11-02,1,Manchester City,L,0,0,1


In [ ]:
def round_to_elo_date(dt):
    return pd.Timestamp(dt.year, dt.month, 1) if dt.day < 15 else pd.Timestamp(dt.year, dt.month, 15)
df_2['merge_date'] = df_2['Date'].apply(round_to_elo_date)
df_elo['merge_date'] = df_elo['date']
name_map_2 = {
    'Arsenal':           'Arsenal',
    'Aston Villa':       'Aston Villa',
    'Burnley':           'Burnley',
    'Chelsea':           'Chelsea',
    'Crystal Palace':    'Crystal Palace',
    'Everton':           'Everton',
    'Hull City':         'Hull',
    'Leicester City':    'Leicester',
    'Liverpool':         'Liverpool',
    'Manchester City':   'Man City',
    'Newcastle Utd':     'Newcastle',
    'QPR':               'QPR',
    'Southampton':       'Southampton',
    'Stoke City':        'Stoke',
    'Sunderland':        'Sunderland',
    'Swansea City':      'Swansea',
    'Tottenham':         'Tottenham',
    'West Brom':         'West Brom',
    'West Ham':          'West Ham',
}
df_2['elo_opponent'] = df_2['Opponent'].map(name_map_2)
df_2['elo_club'] = 'Man United'
df_2 = df_2.merge(
    df_elo[['club', 'merge_date', 'elo']].rename(columns={'elo': 'opp_elo'}),
    left_on=['elo_opponent', 'merge_date'],
    right_on=['club', 'merge_date'],
    how='left'
).drop(columns='club')
df_2 = df_2.merge(
    df_elo[['club', 'merge_date', 'elo']].rename(columns={'elo': 'united_elo'}),
    left_on=['elo_club', 'merge_date'],
    right_on=['club', 'merge_date'],
    how='left'
).drop(columns=['club', 'elo_club'])

In [ ]:
df_2.drop(columns=['merge_date','elo_opponent','opp_elo_y','united_elo_y'], inplace=True, errors='ignore')
df_2

In [ ]:
df_2['elo_diff'] = df_2['united_elo_x'] - df_2['opp_elo_x']
df_2['date'] = pd.to_datetime(df_2['Date'],dayfirst=True)
df_2 = df_2.sort_values('date')
df_2['days_since_last_game_united'] = (
    df_2['date']
      .diff()               # current_date - previous_date
      .dt.days              # convert to integer days
      .fillna(0)            # first match has no “previous” → 0
      .astype(int)
)
df_2.drop(columns='date', inplace=True, errors='ignore')
df_2

,matchday,Date,home_0_away_1,Opponent,Result,points_won,GF,GA,opp_elo_x,united_elo_x,elo_diff,days_since_last_game_united
0,1,2014-08-16,0,Swansea City,L,0,1,2,1672.92,1858.60,185.68,0
1,2,2014-08-24,1,Sunderland,D,1,1,1,1647.63,1858.60,210.97,8
2,3,2014-08-30,1,Burnley,D,1,0,0,1613.48,1858.60,245.12,6
3,4,2014-09-14,0,QPR,W,3,4,0,1570.30,1833.37,263.07,15
4,5,2014-09-21,1,Leicester City,L,0,3,5,1653.65,1837.33,183.68,7
5,6,2014-09-27,0,West Ham,W,3,2,1,1655.79,1837.33,181.54,6
6,7,2014-10-05,0,Everton,W,3,2,1,1808.95,1819.52,10.57,8
7,8,2014-10-20,1,West Brom,D,1,2,2,1632.27,1821.53,189.26,15
8,9,2014-10-26,0,Chelsea,D,1,1,1,1937.44,1821.53,-115.91,6
9,10,2014-11-02,1,Manchester City,L,0,0,1,1920.19,1818.42,-101.77,7


In [ ]:
df_bet_2 = pd.read_csv('/content/14-15 Odds.csv')
df_bet_2['Date'] = pd.to_datetime(df_bet_2['Date'], dayfirst=True)

In [ ]:
df_2['date'] = pd.to_datetime(df_2['Date'], dayfirst=True)
home_odds_map = df_bet_2.set_index(['Date','HomeTeam'])['B365H'].to_dict()
away_odds_map = df_bet_2.set_index(['Date','AwayTeam'])['B365A'].to_dict()

def get_united_win_odds(row):
    key = (row['date'], 'Man United')
    if row['home_0_away_1'] == 0:
        return home_odds_map.get(key, pd.NA)
    else:
        return away_odds_map.get(key, pd.NA)
df_2['united_win_odds'] = df_2.apply(get_united_win_odds, axis=1)

In [ ]:
df_2.drop(columns=['date'], inplace=True, errors='ignore')
df_2

,matchday,Date,home_0_away_1,Opponent,Result,points_won,GF,GA,opp_elo_x,united_elo_x,elo_diff,days_since_last_game_united,united_win_odds
0,1,2014-08-16,0,Swansea City,L,0,1,2,1672.92,1858.60,185.68,0,1.36
1,2,2014-08-24,1,Sunderland,D,1,1,1,1647.63,1858.60,210.97,8,1.75
2,3,2014-08-30,1,Burnley,D,1,0,0,1613.48,1858.60,245.12,6,1.62
3,4,2014-09-14,0,QPR,W,3,4,0,1570.30,1833.37,263.07,15,1.29
4,5,2014-09-21,1,Leicester City,L,0,3,5,1653.65,1837.33,183.68,7,1.67
5,6,2014-09-27,0,West Ham,W,3,2,1,1655.79,1837.33,181.54,6,1.40
6,7,2014-10-05,0,Everton,W,3,2,1,1808.95,1819.52,10.57,8,1.67
7,8,2014-10-20,1,West Brom,D,1,2,2,1632.27,1821.53,189.26,15,1.75
8,9,2014-10-26,0,Chelsea,D,1,1,1,1937.44,1821.53,-115.91,6,2.88
9,10,2014-11-02,1,Manchester City,L,0,0,1,1920.19,1818.42,-101.77,7,4.33


In [ ]:
final_pos_2 = {
    'Chelsea':              1,
    'Manchester City':      2,
    'Arsenal':              3,
    'Tottenham':            5,
    'Liverpool':            6,
    'Southampton':          7,
    'Swansea City':         8,
    'Stoke City':           9,
    'Crystal Palace':      10,
    'Everton':             11,
    'West Ham':            12,
    'West Brom':           13,
    'Leicester City':      14,
    'Newcastle Utd':       15,
    'Sunderland':          16,
    'Aston Villa':         17,
    'Hull City':           18,
    'Burnley':             19,
    'QPR':                 20
}
df_2['opp_final_pos'] = df_2['Opponent'].map(final_pos_2)

In [ ]:
df_2

,matchday,Date,home_0_away_1,Opponent,Result,points_won,GF,GA,opp_elo_x,united_elo_x,elo_diff,days_since_last_game_united,united_win_odds,opp_final_pos
0,1,2014-08-16,0,Swansea City,L,0,1,2,1672.92,1858.60,185.68,0,1.36,8
1,2,2014-08-24,1,Sunderland,D,1,1,1,1647.63,1858.60,210.97,8,1.75,16
2,3,2014-08-30,1,Burnley,D,1,0,0,1613.48,1858.60,245.12,6,1.62,19
3,4,2014-09-14,0,QPR,W,3,4,0,1570.30,1833.37,263.07,15,1.29,20
4,5,2014-09-21,1,Leicester City,L,0,3,5,1653.65,1837.33,183.68,7,1.67,14
5,6,2014-09-27,0,West Ham,W,3,2,1,1655.79,1837.33,181.54,6,1.40,12
6,7,2014-10-05,0,Everton,W,3,2,1,1808.95,1819.52,10.57,8,1.67,11
7,8,2014-10-20,1,West Brom,D,1,2,2,1632.27,1821.53,189.26,15,1.75,13
8,9,2014-10-26,0,Chelsea,D,1,1,1,1937.44,1821.53,-115.91,6,2.88,1
9,10,2014-11-02,1,Manchester City,L,0,0,1,1920.19,1818.42,-101.77,7,4.33,2


In [ ]:
df_2['days_since_last_game_opp'] = [0, 8, 7, 15, 8, 7, 8, 16, 8, 8, 5, 13, 6, 3, 5, 8, 7, 5,2,4, 10, 7, 14, 8, 3, 10, 7, 4, 8, 6, 14, 6, 6, 8, 7, 6, 6, 8]

In [ ]:
df_2

,matchday,Date,home_0_away_1,Opponent,Result,points_won,GF,GA,opp_elo_x,united_elo_x,elo_diff,days_since_last_game_united,united_win_odds,opp_final_pos,days_since_last_game_opp
0,1,2014-08-16,0,Swansea City,L,0,1,2,1672.92,1858.60,185.68,0,1.36,8,0
1,2,2014-08-24,1,Sunderland,D,1,1,1,1647.63,1858.60,210.97,8,1.75,16,8
2,3,2014-08-30,1,Burnley,D,1,0,0,1613.48,1858.60,245.12,6,1.62,19,7
3,4,2014-09-14,0,QPR,W,3,4,0,1570.30,1833.37,263.07,15,1.29,20,15
4,5,2014-09-21,1,Leicester City,L,0,3,5,1653.65,1837.33,183.68,7,1.67,14,8
5,6,2014-09-27,0,West Ham,W,3,2,1,1655.79,1837.33,181.54,6,1.40,12,7
6,7,2014-10-05,0,Everton,W,3,2,1,1808.95,1819.52,10.57,8,1.67,11,8
7,8,2014-10-20,1,West Brom,D,1,2,2,1632.27,1821.53,189.26,15,1.75,13,16
8,9,2014-10-26,0,Chelsea,D,1,1,1,1937.44,1821.53,-115.91,6,2.88,1,8
9,10,2014-11-02,1,Manchester City,L,0,0,1,1920.19,1818.42,-101.77,7,4.33,2,8


In [ ]:
df_2.rename(columns={'opp_elo_x':'opp_elo', 'united_elo_x':'united_elo'}, inplace=True)

In [ ]:
df.rename(columns={'days_since_last_opp_game':'days_since_last_game_opp'}, inplace=True)

In [ ]:
df

,matchday,Date,home_0_away_1,Opponent,Result,points_won,GF,GA,united_elo,opp_elo,elo_diff,united_win_odds,opp_final_pos,days_since_last_game_united,days_since_last_game_opp
0,1,2013-08-17,1,Swansea City,W,3,4,1,1911.40,1665.55,245.85,2.00,12,0,0
1,2,2013-08-26,0,Chelsea,D,1,0,0,1911.40,1887.22,24.18,2.45,3,9,8
2,3,2013-09-01,1,Liverpool,L,0,0,1,1921.33,1801.01,120.32,2.88,2,6,8
3,4,2013-09-14,0,Crystal Palace,W,3,2,0,1921.33,1525.75,395.58,1.22,11,13,14
4,5,2013-09-22,1,Manchester City,L,0,1,4,1911.61,1866.28,45.33,3.60,1,8,8
5,6,2013-09-28,0,West Brom,L,0,1,2,1911.61,1654.51,257.10,1.30,17,6,7
6,7,2013-10-05,1,Sunderland,W,3,2,1,1900.68,1614.09,286.59,1.65,14,7,6
7,8,2013-10-19,0,Southampton,D,1,1,1,1907.86,1682.13,225.73,1.57,8,14,13
8,9,2013-10-26,0,Stoke City,W,3,3,2,1907.86,1658.97,248.89,1.33,9,7,7
9,10,2013-11-02,1,Fulham,W,3,3,1,1909.41,1671.13,238.28,1.57,19,7,7


In [ ]:
df_2 = df_2[['matchday','Date','home_0_away_1','Opponent',	'Result',	'points_won'	,'GF'	,'GA'	,'united_elo'	,'opp_elo',	'elo_diff',	'united_win_odds'	,'opp_final_pos','days_since_last_game_united','days_since_last_game_opp']]

In [ ]:
df_2

,matchday,Date,home_0_away_1,Opponent,Result,points_won,GF,GA,united_elo,opp_elo,elo_diff,united_win_odds,opp_final_pos,days_since_last_game_united,days_since_last_game_opp
0,1,2014-08-16,0,Swansea City,L,0,1,2,1858.60,1672.92,185.68,1.36,8,0,0
1,2,2014-08-24,1,Sunderland,D,1,1,1,1858.60,1647.63,210.97,1.75,16,8,8
2,3,2014-08-30,1,Burnley,D,1,0,0,1858.60,1613.48,245.12,1.62,19,6,7
3,4,2014-09-14,0,QPR,W,3,4,0,1833.37,1570.30,263.07,1.29,20,15,15
4,5,2014-09-21,1,Leicester City,L,0,3,5,1837.33,1653.65,183.68,1.67,14,7,8
5,6,2014-09-27,0,West Ham,W,3,2,1,1837.33,1655.79,181.54,1.40,12,6,7
6,7,2014-10-05,0,Everton,W,3,2,1,1819.52,1808.95,10.57,1.67,11,8,8
7,8,2014-10-20,1,West Brom,D,1,2,2,1821.53,1632.27,189.26,1.75,13,15,16
8,9,2014-10-26,0,Chelsea,D,1,1,1,1821.53,1937.44,-115.91,2.88,1,6,8
9,10,2014-11-02,1,Manchester City,L,0,0,1,1818.42,1920.19,-101.77,4.33,2,7,8


In [ ]:
df["season"] = "2013/14"
df_2["season"] = "2014/15"
final_df = pd.concat([df, df_2], ignore_index=True)

In [ ]:
final_df

,matchday,Date,home_0_away_1,Opponent,Result,points_won,GF,GA,united_elo,opp_elo,elo_diff,united_win_odds,opp_final_pos,days_since_last_game_united,days_since_last_game_opp,season
0,1,2013-08-17,1,Swansea City,W,3,4,1,1911.40,1665.55,245.85,2.00,12,0,0,2013/14
1,2,2013-08-26,0,Chelsea,D,1,0,0,1911.40,1887.22,24.18,2.45,3,9,8,2013/14
2,3,2013-09-01,1,Liverpool,L,0,0,1,1921.33,1801.01,120.32,2.88,2,6,8,2013/14
3,4,2013-09-14,0,Crystal Palace,W,3,2,0,1921.33,1525.75,395.58,1.22,11,13,14,2013/14
4,5,2013-09-22,1,Manchester City,L,0,1,4,1911.61,1866.28,45.33,3.60,1,8,8,2013/14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,34,2015-04-26,1,Everton,L,0,0,3,1855.69,1722.47,133.22,2.15,11,8,8,2014/15
72,35,2015-05-02,0,West Brom,L,0,0,1,1832.55,1607.55,225.00,1.36,13,6,7,2014/15
73,36,2015-05-09,1,Crystal Palace,W,3,2,1,1832.55,1633.15,199.40,1.91,10,7,6,2014/15
74,37,2015-05-17,0,Arsenal,D,1,1,1,1822.84,1860.18,-37.34,2.45,3,8,6,2014/15


In [ ]:
# Finally gotten here - YAYYY #
final_df.to_csv('united_pt1.csv',index=False)